# 0. Load And Concatenate The Datasets

In [32]:
import pandas as pd
import numpy as np
import os
import re

# Define file paths for clarity
# ATTENTION: The files are assumed to be .xlsx under data/raw/
DATA_PATH = "data/raw/" 

# List of files to load and process (both features and scores, longitudinal and baseline)
FEATURE_FILES = ["BaselineData.xlsx", "LongitData.xlsx"]
SCORE_FILES = ["MCIscore_Baseline.xlsx", "MCIscore_Longit_V1.xlsx", "MCIscore_Longit_V2.xlsx"]

# --- 0. Load and Consolidate Raw Data Files ---
print("# 0. Loading and Consolidating Raw Data Files")

# 0.1. Consolidate Feature Data (Baseline and Longitudinal)
all_features = []
print("Consolidating Feature Files...")
for f in FEATURE_FILES:
    file_path = os.path.join(DATA_PATH, f)
    df = pd.read_excel(file_path)
    df.columns = [c.strip() for c in df.columns]  # normalize column names
    all_features.append(df)
    print(f"Loaded {f}: {df.shape[0]} rows.")

# Concatenate all feature data into a single DataFrame
df_all_features = pd.concat(all_features, ignore_index=True, sort=False)
print(f"\nTotal feature rows combined: {df_all_features.shape[0]}")
# Drop duplicates in case a row exists in both BaselineData and LongitData (keep the last/most comprehensive version)
if 'SubjectNumber' in df_all_features.columns and 'Visit' in df_all_features.columns:
    df_all_features.drop_duplicates(subset=['SubjectNumber', 'Visit'], keep='last', inplace=True)
    print(f"Unique feature rows after dropping duplicates: {df_all_features.shape[0]}")
else:
    print("Warning: 'SubjectNumber' or 'Visit' not found in feature files.")

# 0.2. Consolidate Score Data (Baseline and Longitudinal) with robust column detection
score_dfs = []
print("\nConsolidating Score Files (MCI Scores)...")
for f in SCORE_FILES:
    file_path = os.path.join(DATA_PATH, f)
    df = pd.read_excel(file_path)
    df.columns = [c.strip() for c in df.columns]
    df['ScoreFile'] = f
    score_dfs.append(df)
    print(f"Loaded {f}: {df.shape[0]} rows.")

# Concatenate score files
df_all_scores = pd.concat(score_dfs, ignore_index=True, sort=False)
print(f"Total score rows combined: {df_all_scores.shape[0]}")

# Helper: detect SubjectNumber and Visit columns (case-insensitive)
def detect_column(df, patterns):
    for p in patterns:
        for c in df.columns:
            if re.search(p, c, re.I):
                return c
    return None

sub_col = detect_column(df_all_scores, ['subject', '^id$'])
visit_col = detect_column(df_all_scores, ['visit', 'timepoint', 'session'])
if sub_col is None or visit_col is None:
    raise ValueError("Could not detect SubjectNumber or Visit columns in score files.")
# Normalize column names
df_all_scores.rename(columns={sub_col: 'SubjectNumber', visit_col: 'Visit'}, inplace=True)

# Detect the MCI score column (look for column names containing 'mci' and/or 'score')
score_candidates = [c for c in df_all_scores.columns if re.search('mci.*score|score.*mci|mci_score|mci', c, re.I)]
if not score_candidates:
    # fallback: any numeric-ish column other than subject/visit/ScoreFile
    possible = [c for c in df_all_scores.columns if c not in ['SubjectNumber','Visit','ScoreFile']]
    numeric_candidates = []
    for c in possible:
        try:
            pd.to_numeric(df_all_scores[c].dropna().iloc[:10])
            numeric_candidates.append(c)
        except Exception:
            pass
    if numeric_candidates:
        mci_col = numeric_candidates[0]
        print(f"Warning: Using numeric column '{mci_col}' as MCI score (no explicit 'mci' found).")
    else:
        raise ValueError("Could not find an MCI score column in score files.")
else:
    # prefer candidate containing 'score' word
    mci_col = next((c for c in score_candidates if re.search('score', c, re.I)), score_candidates[0])

# Ensure score is numeric
df_all_scores[mci_col] = pd.to_numeric(df_all_scores[mci_col], errors='coerce')
print(f"Detected MCI score column: {mci_col}")

# Attach source groups: baseline rows and V2 rows (filename-based)
df_base_rows = df_all_scores[df_all_scores['ScoreFile'].str.contains('Baseline', case=False, na=False)].copy()
df_v2_rows = df_all_scores[df_all_scores['ScoreFile'].str.contains('V2', case=False, na=False) | df_all_scores['ScoreFile'].str.contains('V_2', case=False, na=False)].copy()

# For baseline: keep last non-null per SubjectNumber+Visit
if not df_base_rows.empty:
    df_base_rows.sort_values(['SubjectNumber','Visit'], inplace=True, kind='stable')
    df_base_rows['MCI_score_base'] = df_base_rows[mci_col]
    df_base_agg = df_base_rows.drop_duplicates(subset=['SubjectNumber','Visit'], keep='last')[['SubjectNumber','Visit','MCI_score_base']]
    print(f"Baseline MCI unique pairs: {df_base_agg.shape[0]}")
else:
    df_base_agg = pd.DataFrame(columns=['SubjectNumber','Visit','MCI_score_base'])
    print("No baseline score rows detected.")

# For V2: same approach
if not df_v2_rows.empty:
    df_v2_rows.sort_values(['SubjectNumber','Visit'], inplace=True, kind='stable')
    df_v2_rows['MCI_score_v2'] = df_v2_rows[mci_col]
    df_v2_agg = df_v2_rows.drop_duplicates(subset=['SubjectNumber','Visit'], keep='last')[['SubjectNumber','Visit','MCI_score_v2']]
    print(f"V2 MCI unique pairs: {df_v2_agg.shape[0]}")
else:
    df_v2_agg = pd.DataFrame(columns=['SubjectNumber','Visit','MCI_score_v2'])
    print("No V2 score rows detected.")

# Drop duplicate score rows in the master score table to keep it clean
df_all_scores.drop_duplicates(subset=['SubjectNumber','Visit', mci_col], keep='last', inplace=True)
print(f"Unique score rows after dropping exact duplicates: {df_all_scores.shape[0]}")

# Merge the baseline and v2 score aggregates into the feature table (works for both baseline and longitudinal rows)
if 'SubjectNumber' in df_all_features.columns and 'Visit' in df_all_features.columns:
    df_integrated = df_all_features.merge(df_base_agg, on=['SubjectNumber','Visit'], how='left')
    df_integrated = df_integrated.merge(df_v2_agg, on=['SubjectNumber','Visit'], how='left')
    print(f"Features after merging base and v2 scores: {df_integrated.shape[0]} rows, columns: {df_integrated.shape[1]}")
else:
    print("Cannot merge scores into features: 'SubjectNumber' or 'Visit' missing in features.")

# Summary sample rows to confirm
print("\nSample of merged features with new score columns:")
print(df_integrated[['SubjectNumber','Visit'] + [c for c in ['MCI_score_base','MCI_score_v2'] if c in df_integrated.columns]].head(10))

# Keep objects for downstream cells
# df_all_features, df_all_scores, df_base_agg, df_v2_agg are available for next steps


# 0. Loading and Consolidating Raw Data Files
Consolidating Feature Files...
Loaded BaselineData.xlsx: 82 rows.
Loaded BaselineData.xlsx: 82 rows.
Loaded LongitData.xlsx: 94 rows.

Total feature rows combined: 176
Unique feature rows after dropping duplicates: 129

Consolidating Score Files (MCI Scores)...
Loaded MCIscore_Baseline.xlsx: 82 rows.
Loaded MCIscore_Longit_V1.xlsx: 47 rows.
Loaded MCIscore_Longit_V2.xlsx: 47 rows.
Total score rows combined: 176
Detected MCI score column: MCIscore
Baseline MCI unique pairs: 82
V2 MCI unique pairs: 47
Unique score rows after dropping exact duplicates: 129
Features after merging base and v2 scores: 129 rows, columns: 663

Sample of merged features with new score columns:
   SubjectNumber Visit  MCI_score_base  MCI_score_v2
0            973    V1       -1.952523           NaN
1            952    V1       -1.931261           NaN
2            905    V1       -1.516037           NaN
3            901    V1       -0.999904           NaN
4           

# 1. Diagnostic For Subject Visit Count Matching

In [33]:
# Diagnostic cell: Why do we have 129 rows instead of expected 164 (82 subjects x 2 visits)?
# This will print counts, duplicates, per-subject visit counts and list problematic subjects to inspect.
expected_subjects = 82
expected_rows = expected_subjects * 2

print("--- Basic shapes and expectations ---")
print('Expected subjects:', expected_subjects, 'Expected rows (2 visits each):', expected_rows)
print('df_all_features.shape:', getattr(df_all_features, 'shape', 'n/a'))
print('df_all_scores.shape:', getattr(df_all_scores, 'shape', 'n/a'))
print('df_integrated.shape:', getattr(df_integrated, 'shape', 'n/a'))

print("\n--- Unique subjects & visits ---")
if 'SubjectNumber' in df_integrated.columns:
    print('Unique subjects in integrated:', df_integrated['SubjectNumber'].nunique())
else:
    print('SubjectNumber column missing in df_integrated')
if 'Visit' in df_integrated.columns:
    print('Unique Visit values in integrated:', df_integrated['Visit'].unique())
    print('Visit counts:\n', df_integrated['Visit'].value_counts(dropna=False))
else:
    print('Visit column missing in df_integrated')

print("\n--- Per-subject row counts (in integrated) ---")
if 'SubjectNumber' in df_integrated.columns:
    per_sub = df_integrated.groupby('SubjectNumber').size()
    print('Per-subject rows summary:\n', per_sub.describe())
    print('Subjects with 2 rows:', (per_sub==2).sum())
    print('Subjects with 1 row:', (per_sub==1).sum())
    print('Subjects with >2 rows:', (per_sub>2).sum())
else:
    per_sub = None

print("\n--- Duplicate Subject+Visit combos in integrated ---")
if set(['SubjectNumber','Visit']).issubset(df_integrated.columns):
    dup_count = df_integrated.duplicated(subset=['SubjectNumber','Visit']).sum()
    print('Duplicate Subject+Visit rows (should be 0):', dup_count)
else:
    print('Cannot check duplicates: keys missing')

print("\n--- Missing MCI scores ---")
for col in ['MCI_score_base','MCI_score_v2']:
    if col in df_integrated.columns:
        print(f'{col} missing count:', df_integrated[col].isna().sum())
    else:
        print(f'{col} not present in df_integrated')

print("\n--- Dtypes for key columns (features vs scores) ---")
for df_name, df in [('features', df_all_features), ('scores', df_all_scores), ('integrated', df_integrated)]:
    print(f"\n{df_name} dtypes:")
    for c in ['SubjectNumber','Visit']:
        if c in df.columns:
            print(' ', c, ':', df[c].dtype)
        else:
            print(' ', c, ': MISSING')

print("\n--- List some problematic subjects (only 1 visit or missing scores) ---")
if per_sub is not None:
    single_visit = per_sub[per_sub==1].index.tolist()
    print('Number of subjects with only 1 visit in integrated:', len(single_visit))
    if len(single_visit)>0:
        print('Sample subjects with 1 visit and their rows:')
        display(df_integrated[df_integrated['SubjectNumber'].isin(single_visit)][['SubjectNumber','Visit','MCI_score_base','MCI_score_v2']].head(20))

    # subjects with missing baseline score
    missing_base = df_integrated[df_integrated['MCI_score_base'].isna()]['SubjectNumber'].unique().tolist() if 'MCI_score_base' in df_integrated.columns else []
    print('Subjects with missing MCI_score_base (unique):', len(missing_base))
    if missing_base:
        display(df_all_scores[df_all_scores['SubjectNumber'].isin(missing_base)][['SubjectNumber','Visit','ScoreFile']].head(30))

    missing_v2 = df_integrated[df_integrated['MCI_score_v2'].isna()]['SubjectNumber'].unique().tolist() if 'MCI_score_v2' in df_integrated.columns else []
    print('Subjects with missing MCI_score_v2 (unique):', len(missing_v2))
    if missing_v2:
        display(df_all_scores[df_all_scores['SubjectNumber'].isin(missing_v2)][['SubjectNumber','Visit','ScoreFile']].head(30))

print("\n--- Quick checks completed. Suggestions:")
print('- If many subjects have only 1 visit, check that Visit values are consistently labeled (e.g., 1/2 vs V1/V2).')
print('- Normalize SubjectNumber/Visit formats (strip, lower/upper, consistent dtype) and re-run the merge.')
print('- Inspect score files for unexpected ScoreFile names or missing rows for V2.')

--- Basic shapes and expectations ---
Expected subjects: 82 Expected rows (2 visits each): 164
df_all_features.shape: (129, 661)
df_all_scores.shape: (129, 4)
df_integrated.shape: (129, 663)

--- Unique subjects & visits ---
Unique subjects in integrated: 82
Unique Visit values in integrated: ['V1' 'V2']
Visit counts:
 Visit
V1    82
V2    47
Name: count, dtype: int64

--- Per-subject row counts (in integrated) ---
Per-subject rows summary:
 count    82.000000
mean      1.573171
std       0.497661
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       2.000000
dtype: float64
Subjects with 2 rows: 47
Subjects with 1 row: 35
Subjects with >2 rows: 0

--- Duplicate Subject+Visit combos in integrated ---
Duplicate Subject+Visit rows (should be 0): 0

--- Missing MCI scores ---
MCI_score_base missing count: 47
MCI_score_v2 missing count: 82

--- Dtypes for key columns (features vs scores) ---

features dtypes:
  SubjectNumber : int64
  Visit : object

scores d

,SubjectNumber,Visit,MCI_score_base,MCI_score_v2
0,973,V1,-1.952523,NaN
1,952,V1,-1.931261,NaN
2,905,V1,-1.516037,NaN
3,901,V1,-0.999904,NaN
4,895,V1,-1.000130,NaN
5,893,V1,-1.187530,NaN
6,892,V1,-2.074647,NaN
7,890,V1,-0.849653,NaN
8,887,V1,-1.000126,NaN
9,883,V1,-1.000038,NaN


Subjects with missing MCI_score_base (unique): 47


,SubjectNumber,Visit,ScoreFile
82,966,V1,MCIscore_Longit_V1.xlsx
83,964,V1,MCIscore_Longit_V1.xlsx
84,953,V1,MCIscore_Longit_V1.xlsx
85,951,V1,MCIscore_Longit_V1.xlsx
86,950,V1,MCIscore_Longit_V1.xlsx
87,944,V1,MCIscore_Longit_V1.xlsx
88,943,V1,MCIscore_Longit_V1.xlsx
89,939,V1,MCIscore_Longit_V1.xlsx
90,934,V1,MCIscore_Longit_V1.xlsx
91,933,V1,MCIscore_Longit_V1.xlsx


Subjects with missing MCI_score_v2 (unique): 82


,SubjectNumber,Visit,ScoreFile
0,973,V1,MCIscore_Baseline.xlsx
4,952,V1,MCIscore_Baseline.xlsx
16,905,V1,MCIscore_Baseline.xlsx
19,901,V1,MCIscore_Baseline.xlsx
24,895,V1,MCIscore_Baseline.xlsx
26,893,V1,MCIscore_Baseline.xlsx
27,892,V1,MCIscore_Baseline.xlsx
29,890,V1,MCIscore_Baseline.xlsx
30,887,V1,MCIscore_Baseline.xlsx
31,883,V1,MCIscore_Baseline.xlsx



--- Quick checks completed. Suggestions:
- If many subjects have only 1 visit, check that Visit values are consistently labeled (e.g., 1/2 vs V1/V2).
- Normalize SubjectNumber/Visit formats (strip, lower/upper, consistent dtype) and re-run the merge.
- Inspect score files for unexpected ScoreFile names or missing rows for V2.


## 1.1 Subjects' Visit Report

As a result of the dataset analysis, it was found that only **47 out of 82 subjects** in the `Baseline.xlsx` dataset returned for the **second EEG recording**, and their **MCI scores** were calculated.  
The remaining **35 subjects** participated in the EEG recording **only once**.  
This explains why there are **129 records** in total — calculated as `47*2 + 35 = 129` — instead of the **expected 162 rows** (`82*2 = 162`) in the `df_integrated` dataframe.


# 2. Creating The Target Label Based on MCI Scores (HC vs. MCI Classification)

In [34]:
print("\n# 2. Creating the Target Labels based on MCI Scores (HC vs. MCI)")

# Confirm imports available
import re
import numpy as np

# The threshold for binary classification (Score > threshold => MCI==1)
threshold = 0.0

# Ensure df_integrated exists; if not, try to (re)create from available pieces
if 'df_integrated' not in globals():
    if 'df_all_features' in globals():
        print('Warning: df_integrated missing — attempting to re-create from df_all_features and score aggregates')
        try:
            df_integrated = df_all_features.merge(df_base_agg, on=['SubjectNumber','Visit'], how='left')
            df_integrated = df_integrated.merge(df_v2_agg, on=['SubjectNumber','Visit'], how='left')
            print('Re-created df_integrated successfully.')
        except Exception as e:
            raise RuntimeError('Failed to recreate df_integrated automatically: ' + str(e))
    else:
        raise RuntimeError('df_integrated and df_all_features are both missing — cannot proceed')

# Normalization helpers
def normalize_subject(x):
    if pd.isna(x):
        return np.nan
    return str(x).strip()

def normalize_visit_val(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # try to get first integer (V1, 1, 2, 'Visit 1' etc.)
    m = re.search(r'(\d+)', s)
    if m:
        try:
            return int(m.group(1))
        except Exception:
            return s
    # fallback to raw string trimmed
    return s

# Apply normalization to key dataframes to ensure consistent keys
for df_name, df in [('features', df_all_features), ('scores', df_all_scores), ('integrated', df_integrated)]:
    if 'SubjectNumber' in df.columns:
        df['SubjectNumber'] = df['SubjectNumber'].apply(normalize_subject)
    if 'Visit' in df.columns:
        df['Visit'] = df['Visit'].apply(normalize_visit_val)
    # write back to globals to keep consistency
    globals()[f'df_{df_name}'] = df if df_name!='integrated' else df_integrated

# Now compute Diagnosis labels only where scores are present (leave NaN otherwise)
for col, diag_col in [('MCI_score_base','Diagnosis_base'), ('MCI_score_v2','Diagnosis_v2')]:
    if col in df_integrated.columns:
        df_integrated[diag_col] = np.nan
        mask = df_integrated[col].notna()
        if mask.any():
            df_integrated.loc[mask, diag_col] = (df_integrated.loc[mask, col] > threshold).astype(int)
            print(f"Created '{diag_col}' for {mask.sum()} rows (based on {col}).")
        else:
            print(f"Column {col} exists but all values are NaN — '{diag_col}' left as NaN.")
    else:
        print(f"{col} not found in integrated data — cannot create {diag_col}.")

# Keep sample outputs for inspection
print('\nSample of integrated rows (first 30) to inspect key columns:')
cols_to_show = ['SubjectNumber','Visit','MCI_score_base','Diagnosis_base','MCI_score_v2','Diagnosis_v2']
print([c for c in cols_to_show if c in df_integrated.columns])
display(df_integrated[[c for c in cols_to_show if c in df_integrated.columns]])


# 2. Creating the Target Labels based on MCI Scores (HC vs. MCI)
Created 'Diagnosis_base' for 82 rows (based on MCI_score_base).
Created 'Diagnosis_v2' for 47 rows (based on MCI_score_v2).

Sample of integrated rows (first 30) to inspect key columns:
['SubjectNumber', 'Visit', 'MCI_score_base', 'Diagnosis_base', 'MCI_score_v2', 'Diagnosis_v2']


,SubjectNumber,Visit,MCI_score_base,Diagnosis_base,MCI_score_v2,Diagnosis_v2
0,973,1,-1.952523,0.0,NaN,NaN
1,952,1,-1.931261,0.0,NaN,NaN
2,905,1,-1.516037,0.0,NaN,NaN
3,901,1,-0.999904,0.0,NaN,NaN
4,895,1,-1.000130,0.0,NaN,NaN
...,...,...,...,...,...,...
124,365,2,NaN,NaN,0.006915,1.0
125,363,1,0.344760,1.0,NaN,NaN
126,363,2,NaN,NaN,1.512527,1.0
127,360,1,0.240650,1.0,NaN,NaN
